In [1]:
pip install --quiet opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import time
from pathlib import Path

cascade_path = Path(cv2.data.haarcascades) / "haarcascade_frontalface_default.xml"

if not cascade_path.exists():
    raise FileNotFoundError(f"Could not find Haar cascade at {cascade_path}. "
                            "Try reinstalling opencv-python or download the XML manually.")

face_cascade = cv2.CascadeClassifier(str(cascade_path))
if face_cascade.empty():
    raise RuntimeError("Failed to load Haar cascade. Reinstall OpenCV or provide a valid cascade file.")
print("Haar cascade loaded:", cascade_path)


Haar cascade loaded: C:\Users\aliha\AppData\Local\Programs\Python\Python312\Lib\site-packages\cv2\data\haarcascade_frontalface_default.xml


In [3]:
test_cap = cv2.VideoCapture(0)
ok, frame = test_cap.read()
test_cap.release()

print("Camera OK:", ok)
if ok:
    print("Frame shape:", frame.shape)
else:
    print("If this is False, try changing the camera index in the next cell (0 -> 1 -> 2). "
          "On macOS, you might need to grant Terminal/Jupyter camera access in System Settings.")


Camera OK: True
Frame shape: (480, 640, 3)


In [4]:
CAM_INDEX = 0        # change to 1/2 if you have multiple cameras
FRAME_W, FRAME_H = 640, 480  # you can set (320,240) for lower latency
SCALE_FACTOR = 1.2   # higher = faster but less accurate; 1.1–1.3 are common
MIN_NEIGHBORS = 6    # higher = fewer (but more confident) detections
MIN_SIZE = (60, 60)  # ignore tiny faces

cap = cv2.VideoCapture(CAM_INDEX)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_W)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_H)

if not cap.isOpened():
    raise RuntimeError(f"Could not open camera index {CAM_INDEX}. Try a different index or check permissions.")

cv2.setUseOptimized(True)

fps_avg = 0.0
t_last = time.time()

print("Running... Press ESC or 'q' in the video window to quit.")
while True:
    ok, frame = cap.read()
    if not ok:
        print("Frame grab failed. Stopping.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=SCALE_FACTOR,
        minNeighbors=MIN_NEIGHBORS,
        minSize=MIN_SIZE
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    t_now = time.time()
    dt = t_now - t_last
    t_last = t_now
    inst_fps = 1.0 / dt if dt > 0 else 0.0
    fps_avg = 0.9 * fps_avg + 0.1 * inst_fps if fps_avg > 0 else inst_fps
    cv2.putText(frame, f"Faces: {len(faces)}  FPS: {fps_avg:.1f}",
                (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Realtime Face Detection (ESC/q to quit)", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
print("Stopped and cleaned up.")


Running... Press ESC or 'q' in the video window to quit.
Stopped and cleaned up.


In [ ]:
print("""
If you see no window or an error:
- Try changing CAM_INDEX (0 -> 1 -> 2).
- Close any app that's using the camera (Zoom/Teams/Meet).
- On macOS: grant camera permission to your terminal/Jupyter (System Settings > Privacy & Security > Camera).
- On Linux: ensure your user is in the 'video' group (ls -l /dev/video0).
- On Windows: Settings > Privacy > Camera: allow desktop apps.

If detection is jumpy/slow:
- Lower resolution: set FRAME_W, FRAME_H to (320, 240).
- Increase SCALE_FACTOR toward 1.25–1.3 for speed.
- Increase MIN_NEIGHBORS (e.g., 7–8) to reduce false boxes.
""")
